In [1]:
import glob
import os
import pandas as pd

In [ ]:
data_path='/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-Final/SM_Data'
# collect all the *csv files under the data_path (have subfolders)
csv_files = glob.glob(os.path.join(data_path, '**', '*.csv'), recursive=True)
# 'radar' not in the csv_files
imu_files = [f for f in csv_files if 'radar' not in f]
print('the number of imu files is '+str(len(imu_files)))
imu_files

the number of imu files is 3230


['/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user1/5-1-1/down(LL+RL).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user1/5-1-1/up(LA+RA+C).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user24/2-2-1/up(LA+RA+C).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user24/2-2-1/down(LL+RL).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user24/5-3-1/down(LL+RL).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user24/5-3-1/up(LA+RA+C).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user26/5-1-1/up(LA+RA+C).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user26/5-1-1/down(LL+RL).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/25_watching_TV/user14/2-2-1/down(LL+RL).csv',
 '/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IM

In [3]:
# Rename CSV files: 上(LA+RA+C).csv -> up(LA+RA+C).csv, 下(LL+RL).csv -> down(LL+RL).csv
import shutil

rename_count = 0
for file_path in imu_files:
    dir_path = os.path.dirname(file_path)
    file_name = os.path.basename(file_path)
    
    new_name = None
    if '上(LA+RA+C).csv' in file_name:
        new_name = file_name.replace('上(LA+RA+C).csv', 'up(LA+RA+C).csv')
    elif '下(LL+RL).csv' in file_name:
        new_name = file_name.replace('下(LL+RL).csv', 'down(LL+RL).csv')
    
    if new_name:
        old_path = file_path
        new_path = os.path.join(dir_path, new_name)
        if old_path != new_path and os.path.exists(old_path):
            os.rename(old_path, new_path)
            rename_count += 1
            print(f"Renamed: {file_name} -> {new_name}")

print(f"\nTotal files renamed: {rename_count}")


Total files renamed: 0


In [ ]:
import re

# Create lists to store each component
paths = []
activity_ids = []
user_ids = []
trail_ids = []  # Changed from caption_ids, label_ids, trail_ids
csv_names = []

for file_path in imu_files:
    # Remove the base path and split
    relative_path = file_path.replace('/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/SM/Data/IMU/', '')
    parts = relative_path.split('/')
    
    # Extract components based on new format
    paths.append(file_path)
    activity_ids.append(parts[0])  # e.g., "29squatting"
    user_ids.append(parts[1])      # e.g., "user24"
    trail_ids.append(parts[2]) # e.g., "6-3-3"
    csv_names.append(parts[3])     # e.g., "上(LA+RA+C).csv"

# Create DataFrame
df = pd.DataFrame({
    'path': paths,
    'activity_id': activity_ids,
    'user_id': user_ids,
    'identifier_id': trail_ids,  # Single identifier instead of caption_id, label_id, trail_id
    'name_of_csv': csv_names
})

# Add three new columns
# 1. Split activity_id into activity_id (number) and activity_name (text)
df['activity_id_num'] = df['activity_id'].str.extract(r'(\d+)')
df['activity_name'] = df['activity_id'].str.extract(r'\d+(.*)')

# 2. Extract number from user_id
df['user_id_num'] = df['user_id'].str.extract(r'(\d+)')

# Reorder columns to put new columns after the original ones
column_order = ['path', 'activity_id', 'activity_id_num', 'activity_name', 'user_id', 'user_id_num', 'identifier_id', 'name_of_csv']
df = df[column_order]

# Display the first few rows and basic information about the DataFrame
print("DataFrame shape:", df.shape)
display(df.head())
df.to_csv('imu_files.csv', index=False)

DataFrame shape: (3230, 8)


,path,activity_id,activity_id_num,activity_name,user_id,user_id_num,identifier_id,name_of_csv
0,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,25_watching_TV,25,_watching_TV,user1,1,5-1-1,down(LL+RL).csv
1,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,25_watching_TV,25,_watching_TV,user1,1,5-1-1,up(LA+RA+C).csv
2,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,25_watching_TV,25,_watching_TV,user24,24,2-2-1,up(LA+RA+C).csv
3,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,25_watching_TV,25,_watching_TV,user24,24,2-2-1,down(LL+RL).csv
4,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,25_watching_TV,25,_watching_TV,user24,24,5-3-1,down(LL+RL).csv


In [5]:
# under the same directory, 下(LL+RL).csv and 上(LA+RA+C).csv is a pair, so we need to merge them
# merge the df with the same 'activity_id', 'user_id', 'caption_id', 'label_id', 'trail_id'

df_merged = df.groupby(['activity_id','user_id','activity_id_num', 'user_id_num', 'identifier_id']).agg({
    'name_of_csv': lambda x: {
        'has_up': any(x.str.contains('up')),
        'has_down': any(x.str.contains('down')),
        'up_path': x[x.str.contains('up')].iloc[0] if any(x.str.contains('up')) else None,
        'down_path': x[x.str.contains('down')].iloc[0] if any(x.str.contains('down')) else None
    }
}).reset_index()

# # Convert the dictionary to separate columns
# df_merged = pd.concat([
#     df_merged.drop('name_of_csv', axis=1),
#     pd.json_normalize(df_merged['name_of_csv'])
# ], axis=1)

# # Now you can easily identify incomplete pairs
# incomplete_pairs = df_merged[~(df_merged['has_up'] & df_merged['has_down'])]

# # Print summary of missing files
# print("Total pairs:", len(df_merged))
# print("\nIncomplete pairs:")
# print("Missing '上' (up) files:", len(df_merged[~df_merged['has_up']]))
# print("Missing '下' (down) files:", len(df_merged[~df_merged['has_down']]))

df_merged


,activity_id,user_id,activity_id_num,user_id_num,identifier_id,name_of_csv
0,0_washing_face,user10,0,10,4-2-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
1,0_washing_face,user11,0,11,1-1-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
2,0_washing_face,user11,0,11,4-2-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
3,0_washing_face,user12,0,12,1-1-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
4,0_washing_face,user12,0,12,2-2-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
...,...,...,...,...,...,...
1611,9_pouring_drink,user8,9,8,4-2-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
1612,9_pouring_drink,user8,9,8,7-2-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
1613,9_pouring_drink,user9,9,9,2-1-1,"{'has_up': True, 'has_down': True, 'up_path': ..."
1614,9_pouring_drink,user9,9,9,3-2-1,"{'has_up': True, 'has_down': True, 'up_path': ..."


In [ ]:
# under the same directory, down(LL+RL).csv and up(LA+RA+C).csv is a pair, so we need to merge them
# merge the df with the same 'activity_id','user_id','activity_id_num', 'user_id_num', 'identifier_id'
# read the imu_files_merged.csv
df = pd.read_csv('imu_files.csv')
df_merged = df.groupby(['activity_id','user_id','activity_id_num', 'user_id_num', 'identifier_id']).agg({
    'path': lambda x: {  # Changed from 'name_of_csv' to 'path'
        'has_up': any(x.str.contains('up')),
        'has_down': any(x.str.contains('down')),
        'up_path': x[x.str.contains('up')].iloc[0] if any(x.str.contains('up')) else None,
        'down_path': x[x.str.contains('down')].iloc[0] if any(x.str.contains('down')) else None
    }
}).reset_index()

# Convert the dictionary to separate columns
df_merged = pd.concat([
    df_merged.drop('path', axis=1),  # Changed from 'name_of_csv' to 'path'
    pd.json_normalize(df_merged['path'])  # Changed from 'name_of_csv' to 'path'
], axis=1)

# Now you can easily identify incomplete pairs
incomplete_pairs = df_merged[~(df_merged['has_up'] & df_merged['has_down'])]

# Print summary of missing files
print("Total pairs:", len(df_merged))
print("\nIncomplete pairs:")
print("Missing 'up' (up) files:", len(df_merged[~df_merged['has_up']]))
print("Missing 'down' (down) files:", len(df_merged[~df_merged['has_down']]))

df_merged.to_csv('imu_files_merged.csv', index=False)
df_merged


Total pairs: 1616

Incomplete pairs:
Missing 'up' (up) files: 33
Missing 'down' (down) files: 14


,activity_id,user_id,activity_id_num,user_id_num,identifier_id,has_up,has_down,up_path,down_path
0,0_washing_face,user10,0,10,4-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
1,0_washing_face,user11,0,11,1-1-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
2,0_washing_face,user11,0,11,4-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
3,0_washing_face,user12,0,12,1-1-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
4,0_washing_face,user12,0,12,2-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
...,...,...,...,...,...,...,...,...,...
1611,9_pouring_drink,user8,9,8,4-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
1612,9_pouring_drink,user8,9,8,7-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
1613,9_pouring_drink,user9,9,9,2-1-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...
1614,9_pouring_drink,user9,9,9,3-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...


In [7]:
df_merged = pd.read_csv('imu_files_merged.csv')

def check_data_exists(path):
    if pd.isna(path) or not path:  # Check if path is None, NaN, or empty string
        return False
    try:
        df = pd.read_csv(path)
        return df.shape[0] > 0
    except (FileNotFoundError, pd.errors.EmptyDataError, ValueError):
        return False

# Apply the safer function to check data existence
df_merged['has_data_up'] = df_merged.apply(lambda row: check_data_exists(row['up_path']), axis=1)
df_merged['has_data_down'] = df_merged.apply(lambda row: check_data_exists(row['down_path']), axis=1)

# Print summary of missing files
print("Total pairs:", len(df_merged))
print("\nIncomplete pairs:")
print("Missing 'up' (up) :", len(df_merged[~df_merged['has_data_up']]))
print("Missing 'down' (down) :", len(df_merged[~df_merged['has_data_down']]))

df_merged.to_csv('imu_files_all.csv', index=False)


Total pairs: 1616

Incomplete pairs:
Missing 'up' (up) : 60
Missing 'down' (down) : 53


In [8]:
df_all = pd.read_csv('imu_files_all.csv')

# filter the df_all, only keep the rows where has_data_up is True and has_data_down is True
df_all = df_all[df_all['has_data_up'] & df_all['has_data_down']]

print("Total pairs:", len(df_all))
df_all.to_csv('imu_files_all_complete.csv', index=False)




Total pairs: 1529


In [ ]:
# add column ['WTLA','WTC','WTRA','WTRL','WTLL'] to df_all, 'WTLA','WTC','WTRA' are from up_path, 'WTRL','WTLL' are from down_path. for example, 'WTLA' are the number of rows in the up_path '设备名称' include 'WTLA', 'WTC' are the number of rows in the up_path '设备名称' include 'WTC', 'WTRA' are the number of rows in the up_path '设备名称' include 'WTRA', 'WTRL' are the number of rows in the down_path '设备名称' include 'WTRL', 'WTLL' are the number of rows in the down_path '设备名称' include 'WTLL'

df_all = pd.read_csv('imu_files_all_complete.csv')
def count_device_rows(path, device):
    try:
        df = pd.read_csv(path)
        return len(df[df['device_name'].str.contains(device, na=False)])
    except:
        return 0

# Add columns for up path devices
df_all['WTLA'] = df_all.apply(lambda row: count_device_rows(row['up_path'], 'WTLA'), axis=1)
df_all['WTC'] = df_all.apply(lambda row: count_device_rows(row['up_path'], 'WTC'), axis=1) 
df_all['WTRA'] = df_all.apply(lambda row: count_device_rows(row['up_path'], 'WTRA'), axis=1)

# Add columns for down path devices
df_all['WTRL'] = df_all.apply(lambda row: count_device_rows(row['down_path'], 'WTRL'), axis=1)
df_all['WTLL'] = df_all.apply(lambda row: count_device_rows(row['down_path'], 'WTLL'), axis=1)

# df_all['activity_id_num'] = df_all['activity_id'].str.extract('(\d+)').astype(int)
# df_all['user_id_num'] = df_all['user_id'].str.extract('(\d+)').astype(int)

df_all.to_csv('imu_files_all_complete_with_device_count.csv', index=False)
df_all


,activity_id,user_id,activity_id_num,user_id_num,identifier_id,has_up,has_down,up_path,down_path,has_data_up,has_data_down,WTLA,WTC,WTRA,WTRL,WTLL
0,0_washing_face,user10,0,10,4-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,15,14,14,15,15
1,0_washing_face,user11,0,11,1-1-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,11,10,11,11,11
2,0_washing_face,user11,0,11,4-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,25,25,25,24,25
3,0_washing_face,user12,0,12,1-1-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,10,11,10,9,10
4,0_washing_face,user12,0,12,2-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,26,25,25,25,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524,9_pouring_drink,user8,9,8,4-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,16,15,13,14,15
1525,9_pouring_drink,user8,9,8,7-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,21,22,22,23,22
1526,9_pouring_drink,user9,9,9,2-1-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,21,21,20,21,22
1527,9_pouring_drink,user9,9,9,3-2-1,True,True,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,/aiot-nvme-15T-x2-hk01/siyang/CUHK-X-example/S...,True,True,19,17,21,20,19


In [10]:
# filter out rows where any of WTLA,WTC,WTRA,WTRL,WTLL are 0
df_all = pd.read_csv('imu_files_all_complete_with_device_count.csv')
df_all = df_all[(df_all['WTLA'] > 0) & 
                (df_all['WTC'] > 0) &
                (df_all['WTRA'] > 0) & 
                (df_all['WTRL'] > 0) &
                (df_all['WTLL'] > 0)]
print("Rows remaining after filtering zeros:", len(df_all))
df_all.to_csv('imu_files_all_complete_with_device_count_filtered.csv', index=False)


Rows remaining after filtering zeros: 1437
